In [7]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
result = wiki.take(1)[0]

In [6]:
def pairs_starting_from_word(words, first_word='word'):
    pairs = []
    
    for i, word in enumerate(words[:-1]):
        if (word == first_word):
            pair = '{}_{}'.format(word, words[i+1])
            cnt = 1
            pairs.append((pair, cnt))
        else:
            continue
    return pairs

In [12]:
wiki_lower = wiki.map(lambda words: [x.lower() for x in words])
wiki_pairs = wiki_lower.flatMap(lambda x: pairs_starting_from_word(x,"narodnaya"))
wiki_red = wiki_pairs.reduceByKey(lambda a, b: a + b, numPartitions=16)
wiki_red_sorted = wiki_red.sortByKey()


In [13]:
result = wiki_red_sorted.collect()
for pair, cnt in result:
    print '{}\t{}'.format(pair, cnt)

narodnaya_gazeta	1
narodnaya_volya	9
